In [19]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import datetime
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")

In [20]:
streetlights_data_file_path = 'data/preprocessed/streetlights_data.geojson'
hex_grid_path = 'data/preprocessed/hex_grid.geojson'

In [21]:
def parse_hex_grid():
    gdf = gpd.read_file(hex_grid_path)
    return gdf

In [22]:
hex_grid_gdf = parse_hex_grid()
estimated_utm_crs = hex_grid_gdf.estimate_utm_crs()
hex_grid_gdf_projected = hex_grid_gdf.to_crs(estimated_utm_crs)

In [23]:
def parse_streetlights_data():
    gdf = gpd.read_file(streetlights_data_file_path)
    estimated_utm_crs = gdf.estimate_utm_crs()
    gdf_projected = gdf.to_crs(estimated_utm_crs)
    return gdf_projected

In [24]:
streetlights_gdf = parse_streetlights_data()

In [25]:
def compute_streelight_counts(hex_grid_gdf, streetlights_gdf):
    joined_gdf = gpd.sjoin(streetlights_gdf, hex_grid_gdf, how="left", predicate="within")
    streetlight_counts = joined_gdf.groupby('index_right').size().reset_index(name='Streetlight Count')

    hex_grid_gdf = hex_grid_gdf.reset_index().merge(streetlight_counts, left_on='index', right_on='index_right', how='left').set_index('index')
    hex_grid_gdf['Streetlight Count'] = hex_grid_gdf['Streetlight Count'].fillna(0).astype(int)  # Handle regions with no points
    return hex_grid_gdf

In [26]:
hex_grid_gdf_projected = compute_streelight_counts(hex_grid_gdf_projected, streetlights_gdf)
hex_grid_gdf['Streetlight Count'] = hex_grid_gdf_projected['Streetlight Count']

In [27]:
hex_grid_gdf.to_file('data/preprocessed/hex_grid.geojson', driver="GeoJSON")